conda environment tf2024

# Prosit dataset

https://figshare.com/articles/dataset/ProteomeTools_non_tryptic_-_Prosit_fragmentation_-_Data/12937092

In [3]:
import numpy as np
import h5py

d = h5py.File("/d0/tp/data_transformer/prosit_download/prediction_hcd_train.hdf5", "r")

ls = list(d.keys())
print(ls)

print(len(d['collision_energy']))

print(len(d['intensities_pred']))

x = np.array(d['sequence_integer'][:])
x.shape

y = np.array(d['intensities_raw'][:])
y.shape


['collision_energy', 'collision_energy_aligned_normed', 'collision_energy_normed', 'intensities_pred', 'intensities_raw', 'masses_raw', 'method', 'precursor_charge_onehot', 'rawfile', 'reverse', 'scan_number', 'score', 'sequence_integer', 'spectral_angle']
9937728
9937728


(9937728, 174)

## visualize y

In [ ]:
# Code from (or adapted from) https://github.com/kusterlab/prosit/ Apache License 2.0
# See README.md

import numpy as np
import pyteomics
from pyteomics import mass  # pylint: disable=unused-import

from aiproteomics.e2e import constants
from aiproteomics.e2e import utils


def generate_aa_comp():
    """
    >>> aa_comp = generate_aa_comp()
    >>> aa_comp["M"]
    Composition({'H': 9, 'C': 5, 'S': 1, 'O': 1, 'N': 1})
    >>> aa_comp["Z"]
    Composition({'H': 9, 'C': 5, 'S': 1, 'O': 2, 'N': 1})
    """

    db = pyteomics.mass.mass.Unimod()
    aa_comp = dict(pyteomics.mass.std_aa_comp)
    s = db.by_title("Oxidation")["composition"]
    aa_comp["Z"] = aa_comp["M"] + s
    s = db.by_title("Carbamidomethyl")["composition"]
    aa_comp["C"] = aa_comp["C"] + s
    return aa_comp


aa_comp = generate_aa_comp()


def get_ions():
    x = np.empty(
        [constants.MAX_ION, len(constants.ION_TYPES), constants.MAX_FRAG_CHARGE],
        dtype="|S6",
    )
    for fz in range(constants.MAX_FRAG_CHARGE):
        for fty_i, fty in enumerate(constants.ION_TYPES):
            for fi in range(constants.MAX_ION):
                ion = fty + str(fi + 1)
                if fz > 0:
                    ion += "({}+)".format(fz + 1)
                x[fi, fty_i, fz] = ion
    x.flatten()
    return x


ox_int = constants.ALPHABET["M(ox)"]
c_int = constants.ALPHABET["C"]


def calculate_mods(sequence_integer):
    """
    >>> x = np.array([2, 15, 4, 3, 0, 0])
    >>> calculate_mods(x)
    1
    >>> x = np.array([2, 15, 21, 3, 0, 0])
    >>> calculate_mods(x)
    2
    """
    return len(np.where((sequence_integer == ox_int) | (sequence_integer == c_int))[0])


def generate_mods_string_tuples(sequence_integer):
    list_mods = []
    for mod in [ox_int, c_int]:
        for position in np.where(sequence_integer == mod)[0]:
            if mod == c_int:
                list_mods.append((position, "C", "Carbamidomethyl"))
            elif mod == ox_int:
                list_mods.append((position, "M", "Oxidation"))
            else:
                raise ValueError("cant be true")
    list_mods.sort(key=lambda tup: tup[0])  # inplace
    return list_mods


def generate_mod_strings(sequence_integer):
    """
    >>> x = np.array([1,2,3,1,2,21,0])
    >>> y, z = generate_mod_strings(x)
    >>> y
    '3/1,C,Carbamidomethyl/4,C,Carbamidomethyl/5,M,Oxidation'
    >>> z
    'Carbamidomethyl@C2; Carbamidomethyl@C5; Oxidation@M6'
    """
    list_mods = generate_mods_string_tuples(sequence_integer)
    if len(list_mods) == 0:
        return "0", ""

    returnString_mods = ""
    returnString_modString = ""
    returnString_mods += str(len(list_mods))
    for i, mod_tuple in enumerate(list_mods):
        returnString_mods += "/" + str(mod_tuple[0]) + "," + mod_tuple[1] + "," + mod_tuple[2]
        if i == 0:
            returnString_modString += mod_tuple[2] + "@" + mod_tuple[1] + str(mod_tuple[0] + 1)
        else:
            returnString_modString += (
                "; " + mod_tuple[2] + "@" + mod_tuple[1] + str(mod_tuple[0] + 1)
            )

    return returnString_mods, returnString_modString


def convert_to_speclib(data, fmt='tsv', intensity_scaling=10000, unknown_value_str='NA'):
    """
    Convert the given data frame of prediction data into a speclib format.
    Supported formats are currently 'msp' and 'tsv'
    """

    IONS = get_ions().reshape(174, -1).flatten()

    speclibtxt = ''
    for i in range(data["iRT"].shape[0]):
        # Get the intensity, ion and mass for all positive intensities
        aIntensity = data["intensities_pred"][i]
        sel = np.where(aIntensity > 0)
        aIntensity = aIntensity[sel]
        aIons = IONS[sel]
        aMass = data["masses_pred"][i][sel]

        # Get the original inputs to the fragmentation model
        collision_energy = data["collision_energy_aligned_normed"][i] * 100
        precursor_charge = data["precursor_charge_onehot"][i].argmax() + 1
        sequence_integer = data["sequence_integer"][i]

        # Get the predicted normalized retention time and ion mobility
        # for this sequence (if available)
        if "iRT" in data:
            iRT = np.squeeze(data["iRT"][i])
        else:
            iRT = None

        if "ccs" in data:
            ccs = np.squeeze(data["ccs"][i])
        else:
            ccs = None

        # If available, use the protein id and gene name in the spec lib
        if "protein_id" in data:
            protein_id = data["protein_id"][i]
        else:
            protein_id = unknown_value_str

        if "gene_name" in data:
            gene_name = data["gene_name"][i]
        else:
            gene_name = unknown_value_str

        # Build a predicted Spectrum representation for this sequence
        spec = Spectrum(
            aIntensity,
            collision_energy,
            iRT,
            ccs,
            aMass,
            precursor_charge,
            sequence_integer,
            aIons,
            protein_id,
            gene_name,
            intensity_scaling,
            unknown_value_str
        )

        # Output string representation of the spectrum in the chosen format
        if fmt == 'msp':
            speclibtxt += spec.to_msp()
        elif fmt == 'tsv':
            speclibtxt += spec.to_tsv()
        else:
            raise ValueError('Format not supported. Supported formats are msp and tsv')

    return speclibtxt


class Spectrum:
    def __init__(
        self,
        aIntensity,
        collision_energy,
        iRT,
        ccs,
        aMass,
        precursor_charge,
        sequence_integer,
        aIons,
        protein_id,
        gene_name,
        intensity_scaling,
        unknown_value_str='NA'
    ):
        self.intensity_scaling = intensity_scaling

        self.aIntensity = aIntensity
        self.collision_energy = collision_energy

        if iRT:
            self.iRT = str(iRT)
        else:
            self.iRT = unknown_value_str

        if ccs:
            self.ccs = str(ccs)
        else:
            self.ccs = unknown_value_str

        self.aMass = aMass
        self.precursor_charge = precursor_charge
        self.aIons = aIons
        self.mod, self.mod_string = generate_mod_strings(sequence_integer)
        self.sequence = utils.get_sequence(sequence_integer)
        # amino acid Z which is defined at the toplevel in generate_aa_comp
        self.precursor_mass = pyteomics.mass.calculate_mass(
            self.sequence.replace("M(ox)", "Z"),
            aa_comp=aa_comp,
            ion_type="M",
            charge=int(self.precursor_charge),
        )
        self.protein_id = protein_id
        self.gene_name = gene_name


    def to_msp(self):
        s = "Name: {sequence}/{charge}\nMW: {precursor_mass}\n"
        s += "Comment: Parent={precursor_mass} Collision_energy={collision_energy} "
        s += "Mods={mod} ModString={sequence}//{mod_string}/{charge} "
        s += "iRT={iRT}"
        s += "ccs={ccs}"
        s += "\nNum peaks: {num_peaks}"
        num_peaks = len(self.aIntensity)
        s = s.format(
            sequence=self.sequence.replace("M(ox)", "M"),
            charge=self.precursor_charge,
            precursor_mass=self.precursor_mass,
            collision_energy=np.round(self.collision_energy[0], 0),
            mod=self.mod,
            mod_string=self.mod_string,
            iRT=self.iRT,
            num_peaks=num_peaks,
        )
        for mz, intensity, ion in zip(self.aMass, self.aIntensity, self.aIons):
            s += "\n" + str(mz) + "\t" + str(intensity) + '\t"'
            s += ion.decode("UTF-8").replace("(", "^").replace("+", "") + '/0.0ppm"'
        return s + '\n'


    def to_tsv(self):
        # Remove oxidation notation from sequence string
        peptide_seq = self.sequence.replace("M(ox)", "M")

        s = ''

        # Add rows for each fragment
        for mz, intensity, ion in zip(self.aMass, self.aIntensity, self.aIons):
            # TODO: Make this values->string->values more sane
            ion_str = ion.decode('UTF-8')
            fragment_type = ion_str[0]
            fragment_series_number = ion_str.split('(')[0][1:]
            if '(' in ion_str:
                fragment_charge = ion_str.split('(')[1].split('+')[0] # This is horrible
            else:
                fragment_charge = '1'
            annotation = fragment_type + fragment_series_number + '^' + fragment_charge

            # Scale intensity (often the highest peak is 10000 by convention)
            intensity *= self.intensity_scaling

            # Collect the values for each column describing a particular peak
            row = [
                str(self.precursor_mass), 
                str(mz),
                annotation,
                str(self.protein_id),
                str(self.gene_name),
                peptide_seq,
                self.sequence,
                str(self.precursor_charge),
                str(intensity),
                self.iRT,
                self.ccs,
                fragment_type,
                fragment_charge,
                fragment_series_number,
                'NA'
            ]
            # Add the row to the output string
            s += '\t'.join(row) + '\n'
            
        return s

def get_tsv_format_header():
    tsv_columns = [
        "PrecursorMz",
        "ProductMz",
        "Annotation",
        "ProteinId",
        "GeneName",
        "PeptideSequence",
        "ModifiedPeptideSequence",
        "PrecursorCharge",
        "LibraryIntensity",
        "NormalizedRetentionTime",
        "PrecursorIonMobility",
        "FragmentType",
        "FragmentCharge",
        "FragmentSeriesNumber",
        "FragmentLossType"
    ]
    return '\t'.join(tsv_columns) + '\n'


In [ ]:
c0 = np.array(d['precursor_charge_onehot'])[0]

x0 = x[0]

y0 = y[0]

y0.shape

int2seq = '_ACDEFGHIKLMNPQRSTVWYo'        
int2seqf = lambda x : ''.join([int2seq[c] for c in x if c > 0])
int2seqf(x0)


# prepare

from aiproteomics.e2e import sanitize
from aiproteomics.e2e import tensorize
import pandas as pd

peptide_data = {"sequence_integer" : np.reshape(x0, (1, -1)),
                "precursor_charge_onehot": np.reshape(c0, (1, -1))}                
            
prediction = y0
peptide_data["intensities_pred"] = prediction
#peptide_data = sanitize.sanitize_prediction_output(peptide_data)
intensities = np.reshape(y0, (1, -1))
intensities[intensities < 0] = 0
intensities = sanitize.normalize_base_peak(intensities)

intensities = sanitize.reshape_dims(intensities)
print(intensities.shape)

sequence_lengths = np.count_nonzero(peptide_data["sequence_integer"], axis=1)
intensities = sanitize.mask_outofrange(intensities, sequence_lengths)

charges = list(peptide_data["precursor_charge_onehot"].argmax(axis=1) + 1)
intensities = sanitize.mask_outofcharge(intensities, charges)

# Shape = N_sequences x 174
intensities = sanitize.reshape_flat(intensities)
peptide_data["intensities_pred"] = intensities

peptide_data["iRT"] = np.array([0])
peptide_data["ccs"] = np.array([0])
peptide_data["collision_energy_aligned_normed"] = np.array([0])

df = pd.DataFrame.from_dict({"modified_sequence" : [int2seqf(x0)], "precursor_charge" : charges})

masses_pred = tensorize.get_mz_applied(df)

from aiproteomics.e2e.constants import MAX_FRAG_CHARGE, MAX_NLOSSES
masses_pred = sanitize.cap(masses_pred, MAX_NLOSSES, MAX_FRAG_CHARGE)

# Calculate length of each (integer) peptide sequence
lengths = (peptide_data["sequence_integer"] > 0).sum(1)
masses_pred = sanitize.mask_outofrange(masses_pred, lengths)
masses_pred = sanitize.mask_outofcharge(masses_pred, df.precursor_charge)


# Output shape: N_sequences x 174
masses_pred = sanitize.reshape_flat(masses_pred)
peptide_data["masses_pred"] = masses_pred

#convert_to_speclib

from aiproteomics.e2e import speclib_writers
speclibtxt = speclib_writers.convert_to_speclib(peptide_data, fmt='tsv')

#plot


In [ ]:
# end here

aMass = data["masses_pred"][i][sel]
IONS = get_ions().reshape(174, -1).flatten()

# Get the intensity, ion and mass for all positive intensities
aIntensity = y0
sel = np.where(aIntensity > 0)
aIntensity = aIntensity[sel]
aIons = IONS[sel]
aMass = peptide_data["masses_pred"][0][sel]


# MS/MS plot

In [ ]:
list(peptide_data)